In [1]:
# Example of LSTM to learn a sequence
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation
from keras.layers import LSTM
import matplotlib.pyplot as plt
import matplotlib
import time
import pandas as pd
import numpy as np
%matplotlib qt

Using TensorFlow backend.


In [32]:
sin = 'sinwave.csv'
df = pd.read_csv(sin,header=0,squeeze=True)
# create X/y pairs
df = concat([df.shift(1), df], axis=1)
df.dropna(inplace=True)
# convert to LSTM friendly format
values = df.values
# split data into train and test
train_size = int(len(df) * 0.67)
test_size = len(df) - train_size
X,Y = values[:,0],values[:,1]
# train, test = series[0:train_size], series[train_size:len(series)]
trainX, trainY = X[0:train_size], Y[0:train_size]
testX, testY = X[-test_size:len(values)], Y[-test_size:len(values)][:-90]
trainX,testX = trainX.reshape(len(trainX), 1, 1), testX[:-90].reshape(len(testX[:-90]), 1, 1)
shifttestY = testY[:-10]
shifttestX = testX[:-10]

In [36]:
# plt.figure()
# # testX = testX[:-93,0].reshape(len(testX[:-93,0]), 1, 1)
# # testY = testY[:-93]
# plt.plot(testY)
# print len(testY)
# plt.plot(shifttestY)
# print len(shifttestY)
# plt.show()
shifttestX.shape

(1550, 1, 1)

In [4]:
# reader for jakobs
seq_len = 1
def load_data(filename, seq_len, normalise_window):
    f = open(filename, 'rb').read()
    data = f.decode().split('\n')

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    
    if normalise_window:
        result = normalise_windows(result)

    result = np.array(result)

    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    return [x_train, y_train, x_test, y_test]

def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data
# X_train, y_train, X_test, y_test = load_data('sp500.csv', seq_len, True)
trainX, trainY, testX, testY = load_data('sp500.csv', seq_len, True)
shifttestY = testY[:-10]

In [ ]:
# create sequence
length = 1000
sequence = [i/float(length) for i in range(length)]
# print(sequence)
# create X/y pairs
df = DataFrame(sequence)
df = concat([df.shift(1), df], axis=1)
df.dropna(inplace=True)
# convert to LSTM friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
trainX = X.reshape(len(X), 1, 1)
trainY = y
testX = trainX
testY = y
# print trainX
# print trainX.shape
print "Done with data preparation"

In [46]:
# def plot_results(predicted_data, true_data):
#     fig = plt.figure(facecolor='white')
#     ax = fig.add_subplot(111)
#     ax.plot(true_data, label='True Data')
#     plt.plot(predicted_data, label='Prediction')
#     plt.legend()
#     plt.show()
models = {}
# trainX = X_train
# trainY = y_train
# testX = X_test
# testY = y_test
for i in range(0,3):
    state = i
    if state == 0:
        print "Fitting stateless model..."
        d0 = time.time()
        # 1. define network STATELESS
        model = Sequential()
        model.add(LSTM(10, input_shape=(1,1)))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        # 2. compile network
        model.compile(optimizer='adam', loss='mean_squared_error')
        # 3. fit network
        history = model.fit(trainX, trainY, epochs=1000, batch_size=len(trainX), verbose=1,validation_split=0.05)
        # summarize history for loss
        plt.figure()
        plt.plot(history.history['loss'],label='loss')
        plt.plot(history.history['val_loss'],label='val_loss')
        plt.title('model loss stateless')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()
        # 4. evaluate network
        loss = model.evaluate(testX, testY, verbose=0)
        print(loss)
        models['stateless'] = model
        d1 = time.time()
        dt = d1-d0
        print "Time fitting stateless: {}s".format(dt)
    elif state == 1:
        print "Fitting stateful model..."
        epoch = 100
        d0 = time.time()
        # 1. define network STATEFUL
        batch_size = 1
        model = Sequential()
        model.add(LSTM(10,batch_input_shape=(batch_size, 1, 1), stateful=True))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        # 2. compile network
        model.compile(optimizer='adam', loss='mean_squared_error')
        # 3. fit network
        loss = []
        val_loss = []
        for i in range(epoch):
            print "{}/{} epoch".format(i+1,epoch)
            history = model.fit(trainX, trainY, epochs=1, batch_size=batch_size,verbose=2,shuffle=False,validation_split=0.05)
            loss.append(history.history['loss'])
            val_loss.append(history.history['val_loss'])
            model.reset_states()
        # summarize history for loss
        plt.figure()
        plt.plot(loss,label='loss')
        plt.plot(val_loss,label='val_loss')
        plt.title('model loss stateful')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()
        models['stateful'] = model
        d1 = time.time()
        dt = d1-d0
        print "Time fitting stateless: {}s".format(dt)
    elif state == 2:
        print "Fitting jakobs model..."
        epoch = 100
#         trainX = X_train
#         trainY = y_train
        d0 = time.time()
        # Use jakobs model
        model = Sequential()
        # add hidden lstm with 50 neurons
        model.add(LSTM(
            50,
            input_shape = (1,1),
            return_sequences = True))
        model.add(Dropout(0.2))
        # add another lstm with 100 neurons
        model.add(LSTM(
            100,
            return_sequences=False))
        model.add(Dropout(0.2))
        # output to 1 neuron
        model.add(Dense(1))
        model.add(Activation("linear"))
        # compile model
        model.compile(loss='mse',optimizer='rmsprop')
        # fit model
        history = model.fit(trainX, trainY, batch_size=32, epochs=epoch, validation_split=0.05)
        # summarize history for loss
        plt.figure()
        plt.plot(history.history['loss'],label='loss')
        plt.plot(history.history['val_loss'],label='val_loss')
        plt.title('model loss jakobs')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()   
        # save to models
        models['jakob'] = model
        d1 = time.time()
        dt = d1-d0
        print "Time fitting jakob's: {}s".format(dt)

Fitting stateless model...
Train on 3181 samples, validate on 168 samples
Epoch 1/1000
3181/3181 [==============================] - 0s - loss: 0.4519 - val_loss: 0.4229
Epoch 2/1000
3181/3181 [==============================] - 0s - loss: 0.4526 - val_loss: 0.4218
Epoch 3/1000
3181/3181 [==============================] - 0s - loss: 0.4496 - val_loss: 0.4206
Epoch 4/1000
3181/3181 [==============================] - 0s - loss: 0.4485 - val_loss: 0.4194
Epoch 5/1000
3181/3181 [==============================] - 0s - loss: 0.4474 - val_loss: 0.4182
Epoch 6/1000
3181/3181 [==============================] - 0s - loss: 0.4453 - val_loss: 0.4170
Epoch 7/1000
3181/3181 [==============================] - 0s - loss: 0.4430 - val_loss: 0.4158
Epoch 8/1000
3181/3181 [==============================] - 0s - loss: 0.4422 - val_loss: 0.4147
Epoch 9/1000
3181/3181 [==============================] - 0s - loss: 0.4399 - val_loss: 0.4134
Epoch 10/1000
3181/3181 [==============================] - 0s - loss: 0

3181/3181 [==============================] - 0s - loss: 0.3198 - val_loss: 0.2991
Epoch 86/1000
3181/3181 [==============================] - 0s - loss: 0.3191 - val_loss: 0.2975
Epoch 87/1000
3181/3181 [==============================] - 0s - loss: 0.3176 - val_loss: 0.2959
Epoch 88/1000
3181/3181 [==============================] - 0s - loss: 0.3173 - val_loss: 0.2942
Epoch 89/1000
3181/3181 [==============================] - 0s - loss: 0.3135 - val_loss: 0.2926
Epoch 90/1000
3181/3181 [==============================] - 0s - loss: 0.3114 - val_loss: 0.2909
Epoch 91/1000
3181/3181 [==============================] - 0s - loss: 0.3110 - val_loss: 0.2893
Epoch 92/1000
3181/3181 [==============================] - 0s - loss: 0.3076 - val_loss: 0.2876
Epoch 93/1000
3181/3181 [==============================] - 0s - loss: 0.3069 - val_loss: 0.2860
Epoch 94/1000
3181/3181 [==============================] - 0s - loss: 0.3047 - val_loss: 0.2843
Epoch 95/1000
3181/3181 [=============================

3181/3181 [==============================] - 0s - loss: 0.1791 - val_loss: 0.1608
Epoch 170/1000
3181/3181 [==============================] - 0s - loss: 0.1762 - val_loss: 0.1592
Epoch 171/1000
3181/3181 [==============================] - 0s - loss: 0.1757 - val_loss: 0.1577
Epoch 172/1000
3181/3181 [==============================] - 0s - loss: 0.1765 - val_loss: 0.1561
Epoch 173/1000
3181/3181 [==============================] - 0s - loss: 0.1690 - val_loss: 0.1546
Epoch 174/1000
3181/3181 [==============================] - 0s - loss: 0.1713 - val_loss: 0.1531
Epoch 175/1000
3181/3181 [==============================] - 0s - loss: 0.1689 - val_loss: 0.1516
Epoch 176/1000
3181/3181 [==============================] - 0s - loss: 0.1672 - val_loss: 0.1501
Epoch 177/1000
3181/3181 [==============================] - 0s - loss: 0.1671 - val_loss: 0.1486
Epoch 178/1000
3181/3181 [==============================] - 0s - loss: 0.1656 - val_loss: 0.1470
Epoch 179/1000
3181/3181 [===================

3181/3181 [==============================] - 0s - loss: 0.0724 - val_loss: 0.0564
Epoch 254/1000
3181/3181 [==============================] - 0s - loss: 0.0757 - val_loss: 0.0555
Epoch 255/1000
3181/3181 [==============================] - 0s - loss: 0.0751 - val_loss: 0.0547
Epoch 256/1000
3181/3181 [==============================] - 0s - loss: 0.0717 - val_loss: 0.0538
Epoch 257/1000
3181/3181 [==============================] - 0s - loss: 0.0725 - val_loss: 0.0530
Epoch 258/1000
3181/3181 [==============================] - 0s - loss: 0.0730 - val_loss: 0.0521
Epoch 259/1000
3181/3181 [==============================] - 0s - loss: 0.0723 - val_loss: 0.0513
Epoch 260/1000
3181/3181 [==============================] - 0s - loss: 0.0688 - val_loss: 0.0505
Epoch 261/1000
3181/3181 [==============================] - 0s - loss: 0.0686 - val_loss: 0.0497
Epoch 262/1000
3181/3181 [==============================] - 0s - loss: 0.0686 - val_loss: 0.0489
Epoch 263/1000
3181/3181 [===================

3181/3181 [==============================] - 0s - loss: 0.0374 - val_loss: 0.0144
Epoch 338/1000
3181/3181 [==============================] - 0s - loss: 0.0365 - val_loss: 0.0141
Epoch 339/1000
3181/3181 [==============================] - 0s - loss: 0.0389 - val_loss: 0.0139
Epoch 340/1000
3181/3181 [==============================] - 0s - loss: 0.0360 - val_loss: 0.0137
Epoch 341/1000
3181/3181 [==============================] - 0s - loss: 0.0338 - val_loss: 0.0134
Epoch 342/1000
3181/3181 [==============================] - 0s - loss: 0.0338 - val_loss: 0.0132
Epoch 343/1000
3181/3181 [==============================] - 0s - loss: 0.0353 - val_loss: 0.0130
Epoch 344/1000
3181/3181 [==============================] - 0s - loss: 0.0343 - val_loss: 0.0128
Epoch 345/1000
3181/3181 [==============================] - 0s - loss: 0.0338 - val_loss: 0.0126
Epoch 346/1000
3181/3181 [==============================] - 0s - loss: 0.0351 - val_loss: 0.0125
Epoch 347/1000
3181/3181 [===================

...

3181/3181 [==============================] - 0s - loss: 0.0199 - val_loss: 0.0030
Epoch 842/1000
3181/3181 [==============================] - 0s - loss: 0.0194 - val_loss: 0.0030
Epoch 843/1000
3181/3181 [==============================] - 0s - loss: 0.0201 - val_loss: 0.0030
Epoch 844/1000
3181/3181 [==============================] - 0s - loss: 0.0198 - val_loss: 0.0030
Epoch 845/1000
3181/3181 [==============================] - 0s - loss: 0.0206 - val_loss: 0.0030
Epoch 846/1000
3181/3181 [==============================] - 0s - loss: 0.0183 - val_loss: 0.0029
Epoch 847/1000
3181/3181 [==============================] - 0s - loss: 0.0193 - val_loss: 0.0029
Epoch 848/1000
3181/3181 [==============================] - 0s - loss: 0.0202 - val_loss: 0.0029
Epoch 849/1000
3181/3181 [==============================] - 0s - loss: 0.0190 - val_loss: 0.0030
Epoch 850/1000
3181/3181 [==============================] - 0s - loss: 0.0189 - val_loss: 0.0030
Epoch 851/1000
3181/3181 [===================

3181/3181 [==============================] - 0s - loss: 0.0189 - val_loss: 0.0029
Epoch 926/1000
3181/3181 [==============================] - 0s - loss: 0.0198 - val_loss: 0.0029
Epoch 927/1000
3181/3181 [==============================] - 0s - loss: 0.0189 - val_loss: 0.0029
Epoch 928/1000
3181/3181 [==============================] - 0s - loss: 0.0190 - val_loss: 0.0029
Epoch 929/1000
3181/3181 [==============================] - 0s - loss: 0.0187 - val_loss: 0.0029
Epoch 930/1000
3181/3181 [==============================] - 0s - loss: 0.0196 - val_loss: 0.0029
Epoch 931/1000
3181/3181 [==============================] - 0s - loss: 0.0195 - val_loss: 0.0028
Epoch 932/1000
3181/3181 [==============================] - 0s - loss: 0.0186 - val_loss: 0.0028
Epoch 933/1000
3181/3181 [==============================] - 0s - loss: 0.0198 - val_loss: 0.0028
Epoch 934/1000
3181/3181 [==============================] - 0s - loss: 0.0188 - val_loss: 0.0028
Epoch 935/1000
3181/3181 [===================

5s - loss: 0.0163 - val_loss: 0.0047
9/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
4s - loss: 0.0163 - val_loss: 0.0014
10/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
4s - loss: 0.0157 - val_loss: 0.0021
11/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
4s - loss: 0.0163 - val_loss: 9.2912e-04
12/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0154 - val_loss: 9.5813e-04
13/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0153 - val_loss: 0.0011
14/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0146 - val_loss: 0.0014
15/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0158 - val_loss: 0.0024
16/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0152 - val_loss: 5.3798e-04
17/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0147 - val_lo

5s - loss: 0.0187 - val_loss: 9.3488e-04
85/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
4s - loss: 0.0186 - val_loss: 5.6525e-04
86/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0172 - val_loss: 7.4003e-04
87/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0188 - val_loss: 0.0020
88/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0191 - val_loss: 0.0017
89/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0198 - val_loss: 0.0013
90/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
4s - loss: 0.0192 - val_loss: 0.0016
91/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
5s - loss: 0.0187 - val_loss: 0.0014
92/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
6s - loss: 0.0194 - val_loss: 0.0013
93/100 epoch
Train on 3181 samples, validate on 168 samples
Epoch 1/1
6s - loss: 0.0191 - val_l

3181/3181 [==============================] - 0s - loss: 0.0058 - val_loss: 0.0034
Epoch 68/100
3181/3181 [==============================] - 1s - loss: 0.0057 - val_loss: 0.0026
Epoch 69/100
3181/3181 [==============================] - 1s - loss: 0.0057 - val_loss: 0.0021
Epoch 70/100
3181/3181 [==============================] - 0s - loss: 0.0059 - val_loss: 0.0025
Epoch 71/100
3181/3181 [==============================] - 1s - loss: 0.0057 - val_loss: 0.0027
Epoch 72/100
3181/3181 [==============================] - 0s - loss: 0.0058 - val_loss: 0.0026
Epoch 73/100
3181/3181 [==============================] - 0s - loss: 0.0057 - val_loss: 0.0024
Epoch 74/100
3181/3181 [==============================] - 1s - loss: 0.0056 - val_loss: 0.0024
Epoch 75/100
3181/3181 [==============================] - 1s - loss: 0.0057 - val_loss: 0.0021
Epoch 76/100
3181/3181 [==============================] - 0s - loss: 0.0057 - val_loss: 0.0040
Epoch 77/100
3181/3181 [==============================] - 0s - 

In [ ]:
# # Try new data pred
# import numpy as np
# length = 50
# sequence = np.arange(0.999,1.01,0.001)
# X = sequence
# X = X.reshape(len(X), 1, 1)
# testX = testX[-50:]
# testY = testY[-50:]
history.history

In [4]:
# 5. make predictions
from sklearn.metrics import mean_squared_error
from math import sqrt
for mod in models.keys():
    if mod == 'stateful':
        predictions = models[mod].predict(testX,batch_size=1, verbose=0)
        rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure(0)
        print "Stateful prediction..."
        print testX[-1],testY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Stateful')
        plt.plot(testX[:, 0],label='initial')
        plt.plot(testY,label='shifted true')
        plt.plot(predictions,label='shifted pred')
        plt.legend()
        plt.show()
    elif mod == 'stateless':
        predictions = models[mod].predict(testX, verbose=0)
        rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure(1)
        print "Stateless prediction..."
        print testX[-1],testY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Stateless')
        plt.plot(testX[:, 0],label='initial')
        plt.plot(testY,label='shifted true')
        plt.plot(predictions,label='shifted pred')
        plt.legend()
        plt.show()
    elif mod == 'jakob':
#         testX = X_test
#         testY = y_test
        predictions = models[mod].predict(testX)
#         predictions = np.reshape(predictions, (predictions.size,))
        rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure(3)
        print "Jakob's prediction..."
        print testX[-1],testY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Jakobs')
        plt.plot(testX[:, 0],label='initial')
        plt.plot(testY,label='shifted true')
        plt.plot(predictions,label='shifted pred')
        plt.legend()
        plt.show()
# stateful = True
# if stateful:
#     predictions = model.predict(X,batch_size=1, verbose=0)
# else:
#     predictions = model.predict(X, verbose=0)
# # print(predictions[:, 0][-10:])
# # predictions = predictions[-10:]
# print X[-1],y[-1],predictions[-1][0]
# plt.plot(X[:, 0])
# plt.plot(predictions)

Stateful prediction...
[[ 0.98127848]] 0.991435244 0.987818
RMSE: 0.00361682406668
Stateless prediction...
[[ 0.98127848]] 0.991435244 0.897326
RMSE: 0.0941091322065
Jakob's prediction...
[[ 0.98127848]] 0.991435244 0.97995
RMSE: 0.0114852928281


In [48]:
# 5. make predictions but shift forward n steps
from sklearn.metrics import mean_squared_error
from math import sqrt
n = 10
for mod in models.keys():
    if mod == 'stateful':
        predicted = shifttestX
        for i in range(n):
            predictions = models[mod].predict(predicted.reshape(len(predicted),1,1),batch_size=1, verbose=0)
            predicted = np.append(predicted,predictions[-1])
            rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure()
        print "Stateful prediction..."
        print shifttestX[-1],shifttestY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Stateful')
        plt.plot(shifttestX[:,0],label='initial')
        plt.plot(testY,label='shifted true')
        plt.plot(predicted,label='shifted pred')
        plt.legend()
        plt.show()
    elif mod == 'stateless':
        predicted = shifttestX
        for i in range(n):
            predictions = models[mod].predict(predicted.reshape(len(predicted),1,1), verbose=0)
            predicted = np.append(predicted,predictions[-1])
            rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure()
        print "Stateless prediction..."
        print shifttestX[-1],shifttestY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Stateless')
        plt.plot(shifttestX[:,0],label='initial')
        plt.plot(testY,label='shifted true')
        plt.plot(predicted,label='shifted pred')
        plt.legend()
        plt.show()
    elif mod == 'jakob':
        predicted = shifttestX
        # testX = X_test
        # testY = y_test
        for i in range(n):
            predictions = models[mod].predict(predicted.reshape(len(predicted),1,1))
#             print predictions[-1]
            predicted = np.append(predicted,predictions[-1])
            rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure()
        print "Jakob's prediction..."
        print len(predicted)
        print shifttestX[-1],shifttestY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Jakobs')
        plt.plot(shifttestX[:,0],label='initial')
        plt.plot(testY,label='shifted true')
        plt.plot(predicted,label='shifted pred')
        plt.legend()
        plt.show()

Stateful prediction...
[[ 0.80588467]] 0.841470985 0.248285
RMSE: 0.750061371369
Stateless prediction...
[[ 0.80588467]] 0.841470985 0.641291
RMSE: 0.357055270118
Jakob's prediction...
1560
[[ 0.80588467]] 0.841470985 0.898326
RMSE: 0.100020014686


In [59]:
X = np.array(0.80588467).reshape(1,1,1)
p = models['jakob'].predict(X)
print p
print X

[[ 0.81853259]]
[[[ 0.80588467]]]


Stateful prediction...50
[[ 0.80588467]] 0.841470985 0.828496
RMSE: 0.0129754229234
Stateless prediction...10
[[ 0.80588467]] 0.841470985 0.782955
RMSE: 0.0585164113687

In [ ]:
testX[:,0]

In [ ]:
plt.plot(testY,label='shifted pred')

In [ ]:
from keras.utils import plot_model
plot_model(models['stateful'], to_file='model_stateful.png',show_shapes=True)

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(models['stateful']).create(prog='dot', format='svg'))